# Heat Equation Porting Exercise

The following exercise will walk through the porting of a 3D heat equation code from Fortran to GT4Py.  The cell below contains a simple 3D heat equation code written in Fortran.  

<div class="alert alert-block alert-info">
    <b> Now it's your turn: </b><br>
    (Hint: Make sure that when you modify code you retain the original code by commenting it out so that you undo any of the modifications you do.)
    <ol>
For this part of the exercise, the developer will do the following.

 <li style="margin-bottom: 10px"> Initialize Serialbox in the Fortran code to write to a folder <code>./data</code> </li>
 <li style="margin-bottom: 10px"> Serialize the initial heat data <code>curr_Heat</code> and relevant variables to the Python/GT4Py port.</li>
 <li style="margin-bottom: 10px"> Serialize the output heat data to compare to the Python/GT4Py port </li>

After inserting the Serialbox calls, the developer can run the cell to create a source file called `heat.F90`.
    </ol>
</div>

In [ ]:
%%writefile heat.F90

program heat

    implicit none

    integer :: ii, jj, kk, N, t_steps, curr_step, nx, ny, nz, nhalo

    double precision :: scale

    double precision, dimension(:,:,:), allocatable :: curr_Heat, future_Heat

    write(*,*) "Enter the Number of discretization cells N: "
    read(*,*) N
    write(*,*) "Enter number of time steps : "
    read(*,*) t_steps

    write(*,*) "N = ", N
    write(*,*) "Time steps = ", t_steps

    ! Note : Currently, the default domain is a cube, so the number of 
    !        discretization cells in x, y, and z are the same
    nhalo = 3
    nx = N
    ny = N
    nz = N
    allocate(curr_Heat(nx+2*nhalo,ny+2*nhalo,nz), future_Heat(nx+2*nhalo,ny+2*nhalo,nz))
   
    !***Insert Serialbox calls here for initialization***
    !$ser init directory='./data' prefix="HEAT" unique_id=.true.
    !$ser mode write
    !$ser on

    curr_Heat(:, :, :)   = 0.0
    future_Heat(:, :, :) = 0.0

    scale = 0.1

    curr_Heat(nhalo+1:nx+nhalo, nhalo+1:ny+nhalo, 2:nz-1) = 1.0

    !***Insert Serialbox calls to create a savepoint to save initial starting data***
    !$ser savepoint 'starting_heat'
    !$ser data init_T=curr_Heat scale=scale t_steps=t_steps nhalo=nhalo
    do curr_step = 1,t_steps
        do kk = 2,nz-1
            do jj = nhalo+1,ny+nhalo
                do ii = nhalo+1,nx+nhalo
                    future_Heat(ii,jj,kk) = curr_Heat(ii,jj,kk)              &
                                        + scale * (curr_Heat(ii-1,jj,kk)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii+1,jj,kk))  &
                                        + scale * (curr_Heat(ii,jj-1,kk)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii,jj+1,kk))  &
                                        + scale * (curr_Heat(ii,jj,kk-1)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii,jj,kk+1))
                enddo
            enddo
        enddo

        curr_Heat = future_Heat

    enddo
    
    !***Insert Serialbox calls to create a savepoint to write finalized data***
    !$ser savepoint 'final_heat'
    !$ser data final_T=curr_Heat
    !$ser cleanup

end program

After adding the Serialbox calls, the developer can run the cell below to perform the following:

* Call `pp_ser.py` to substitute the Serialbox "directives" with Serialbox library calls in the Fortran code and output the result into a new Fortran source file called `s_heat.F90`
* Compile `s_heat.F90` and link with the Serialbox libraries into a binary called `HEAT`
* Run the `HEAT` binary 

Note that the code implementation has two inputs (size of the domain in cells and number of time steps) that must be listed on individual lines in the cell below when running the `HEAT` binary.  The developer can try different entries to help with debugging if necessary.

In [ ]:
%%bash

[ -f HEAT ] && rm HEAT
[ -f s_heat.F90 ] && rm s_heat.F90

python3 ${SERIALBOX_ROOT}/python/pp_ser/pp_ser.py -s -v --output=s_heat.F90 heat.F90

gfortran -O3 -cpp -DSERIALIZE -o HEAT s_heat.F90 \
   -I${SERIALBOX_ROOT}/include \
    ${SERIALBOX_ROOT}/lib/libSerialboxFortran.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxC.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxCore.a \
 -lpthread -lstdc++ -lstdc++fs
 rm -rf ./data
./HEAT
40
10



After ensuring that you are able to run the code and successfully serialize the data, you can start to write your Python/GT4Py port of the heat equation code in the cell below.  To compare solutions between the Python and the Fortran, use the `np.array_equal()`  function.

Here are a couple of things to keep in mind.

* Python array indices start at 0
* While it's not explicitly stated, the heat equation example has an implied boundary condition where the "surface" temperatures are set to 0

## Python/GT4py solution

In [ ]:
import os
import sys
sys.path.append(os.environ.get('SERIALBOX_ROOT') + '/python')
import serialbox as ser
import numpy as np
import gt4py.gtscript as gtscript
import gt4py.storage as gt_storage

backend = "numpy"
F_TYPE = np.float64

@gtscript.stencil(backend=backend)
def heat_update(curr_T   : gtscript.Field[F_TYPE],
                future_T : gtscript.Field[F_TYPE],
                *,
                scale    : np.float64):
    with computation(PARALLEL), interval(...):
        future_T = curr_T + scale * (curr_T[-1,0,0] - 2.0*curr_T + curr_T[1,0,0]) \
                          + scale * (curr_T[0,-1,0] - 2.0*curr_T + curr_T[0,1,0]) \
                          + scale * (curr_T[0,0,-1] - 2.0*curr_T + curr_T[0,0,1])

        curr_T = future_T

serializer = ser.Serializer(ser.OpenModeKind.Read, './data', 'HEAT')

sp = serializer.get_savepoint('starting_heat')

curr_T = serializer.read('init_T', sp[0])
scale     = serializer.read('scale', sp[0])[0]
t_steps   = serializer.read('t_steps', sp[0])[0]
nhalo = serializer.read('nhalo', sp[0])[0]
origin=(nhalo,nhalo,1)
full_shape = curr_T.shape
nx = full_shape[0] - 2 * nhalo
ny = full_shape[1] - 2 * nhalo
nz = full_shape[2] - 2
curr_T_gt = gt_storage.from_array(curr_T, 
                                  backend=backend, 
                                  default_origin=(nhalo,nhalo,1))

future_T_gt = gt_storage.zeros(backend=backend, 
                               dtype=F_TYPE, 
                               shape=full_shape,
                               default_origin=origin)

for curr_step in range(t_steps):
    heat_update(curr_T=curr_T_gt,
                future_T=future_T_gt,
                scale=scale,
                origin=origin,
                domain=(nx, ny, nz))

sp = serializer.get_savepoint('final_heat')
final_T = serializer.read('final_T', sp[0])

if np.array_equal(final_T,curr_T_gt):
    print("Solution is valid!")
else:
    raise Exception("Solution is NOT valid")

## Python-only solution

In [ ]:
import os
import sys
sys.path.append(os.environ.get('SERIALBOX_ROOT') + '/python')
import serialbox as ser
import numpy as np

serializer = ser.Serializer(ser.OpenModeKind.Read, './data', 'HEAT')
sp = serializer.get_savepoint('starting_heat')

curr_T = serializer.read('init_T', sp[0])
scale     = serializer.read('scale', sp[0])[0]
t_steps   = serializer.read('t_steps', sp[0])[0]
nhalo   = serializer.read('nhalo', sp[0])[0]
full_shape = curr_T.shape
nx = full_shape[0] - 2 * nhalo
ny = full_shape[1] - 2 * nhalo
nz = full_shape[2] - 2
future_T = np.zeros((curr_T.shape[0],
                        curr_T.shape[1],
                        curr_T.shape[2]),dtype='float64')
for curr_step in range(t_steps):
    for ii in range(nhalo, nhalo + nx):  
        for jj in range(nhalo, nhalo + ny):
            for kk in range(1, nz +1):       
                future_T[ii,jj,kk] = curr_T[ii,jj,kk] \
                  + scale * (curr_T[ii-1,jj,   kk]   - 2.0*curr_T[ii,jj,kk] + curr_T[ii+1,jj,  kk]) \
                  + scale * (curr_T[ii,  jj-1, kk]   - 2.0*curr_T[ii,jj,kk] + curr_T[ii,  jj+1,kk]) \
                  + scale * (curr_T[ii,  jj,   kk-1] - 2.0*curr_T[ii,jj,kk] + curr_T[ii,  jj,  kk+1])
    curr_T[:,:,:] = future_T[:,:,:]
sp = serializer.get_savepoint('final_heat')
final_T = serializer.read('final_T', sp[0])

if np.array_equal(final_T,curr_T):
    print("Solution is valid!")
else:
   
    raise Exception("Solution is NOT valid")

## Single precision
<div class="alert alert-block alert-info">
    <b> Now it's your turn: </b><br>

Try the same problem, but using single precision and  `np.allclose()` for comparison
</div>

In [ ]:
%%writefile heat_single.F90

program heat

    implicit none

    integer :: ii, jj, kk, N, t_steps, curr_step, nx, ny, nz, nhalo

    real :: scale

    real, dimension(:,:,:), allocatable :: curr_Heat, future_Heat

    write(*,*) "Enter the Number of discretization cells N: "
    read(*,*) N
    write(*,*) "Enter number of time steps : "
    read(*,*) t_steps

    write(*,*) "N = ", N
    write(*,*) "Time steps = ", t_steps

    ! Note : Currently, the default domain is a cube, so the number of 
    !        discretization cells in x, y, and z are the same
    nhalo = 3
    nx = N
    ny = N
    nz = N
    allocate(curr_Heat(nx+2*nhalo,ny+2*nhalo,nz), future_Heat(nx+2*nhalo,ny+2*nhalo,nz))
   
    !***Insert Serialbox calls here for initialization***
    !$ser init directory='./data_single' prefix="HEAT_SINGLE" unique_id=.true.
    !$ser mode write
    !$ser on

    curr_Heat(:, :, :)   = 0.0
    future_Heat(:, :, :) = 0.0

    scale = 0.1

    curr_Heat(nhalo+1:nx+nhalo, nhalo+1:ny+nhalo, 2:nz-1) = 1.0

    !***Insert Serialbox calls to create a savepoint to save initial starting data***
    !$ser savepoint 'starting_heat'
    !$ser data init_T=curr_Heat scale=scale t_steps=t_steps nhalo=nhalo
    do curr_step = 1,t_steps
        do kk = 2,nz-1
            do jj = nhalo+1,ny+nhalo
                do ii = nhalo+1,nx+nhalo
                    future_Heat(ii,jj,kk) = curr_Heat(ii,jj,kk)              &
                                        + scale * (curr_Heat(ii-1,jj,kk)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii+1,jj,kk))  &
                                        + scale * (curr_Heat(ii,jj-1,kk)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii,jj+1,kk))  &
                                        + scale * (curr_Heat(ii,jj,kk-1)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii,jj,kk+1))
                enddo
            enddo
        enddo

        curr_Heat = future_Heat

    enddo
    
    !***Insert Serialbox calls to create a savepoint to write finalized data***
    !$ser savepoint 'final_heat'
    !$ser data final_T=curr_Heat
    !$ser cleanup

end program

In [ ]:
%%bash

[ -f HEAT_SINGLE ] && rm HEAT_SINGLE
[ -f s_heat_single.F90 ] && rm s_heat_single.F90

python3 ${SERIALBOX_ROOT}/python/pp_ser/pp_ser.py -s -v --output=s_heat_single.F90 heat_single.F90

gfortran -O3 -cpp -DSERIALIZE -o HEAT_SINGLE s_heat_single.F90 \
   -I${SERIALBOX_ROOT}/include \
    ${SERIALBOX_ROOT}/lib/libSerialboxFortran.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxC.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxCore.a \
 -lpthread -lstdc++ -lstdc++fs
 
rm -rf ./data_single
./HEAT_SINGLE
40
10


## GT4py solution single precision 

In [ ]:
import os
import sys
sys.path.append(os.environ.get('SERIALBOX_ROOT') + '/python')
import serialbox as ser
import gt4py.gtscript as gtscript
import gt4py.storage as gt_storage
import numpy as np

backend = "numpy"
F_TYPE = np.float32

@gtscript.stencil(backend=backend)
def heat_update(curr_T   : gtscript.Field[F_TYPE],
                future_T : gtscript.Field[F_TYPE],
                *,
                scale    : np.float32):
    with computation(PARALLEL), interval(...):
        future_T = curr_T + scale * (curr_T[-1,0,0] - 2.0*curr_T + curr_T[1,0,0]) \
                          + scale * (curr_T[0,-1,0] - 2.0*curr_T + curr_T[0,1,0]) \
                          + scale * (curr_T[0,0,-1] - 2.0*curr_T + curr_T[0,0,1])

        curr_T = future_T

serializer = ser.Serializer(ser.OpenModeKind.Read, './data_single', 'HEAT_SINGLE')

sp = serializer.get_savepoint('starting_heat')

curr_T = serializer.read('init_T', sp[0])
scale     = serializer.read('scale', sp[0])[0]
t_steps   = serializer.read('t_steps', sp[0])[0]
nhalo = serializer.read('nhalo', sp[0])[0]
origin=(nhalo,nhalo,1)
full_shape = curr_T.shape
nx = full_shape[0] - 2 * nhalo
ny = full_shape[1] - 2 * nhalo
nz = full_shape[2] - 2
curr_T_gt = gt_storage.from_array(curr_T, 
                                  backend=backend, 
                                  default_origin=(nhalo,nhalo,1))

future_T_gt = gt_storage.zeros(backend=backend, 
                               dtype=F_TYPE, 
                               shape=full_shape,
                               default_origin=origin)

for curr_step in range(t_steps):
    heat_update(curr_T=curr_T_gt,
                future_T=future_T_gt,
                scale=scale,
                origin=origin,
                domain=(nx, ny, nz))

sp = serializer.get_savepoint('final_heat')
final_T = serializer.read('final_T', sp[0])

if np.array_equal(final_T,curr_T_gt):
    print("Solution is valid!")
else:
    raise Exception("Solution is NOT valid")

## Python only single-precision solution

In [ ]:
import os
import sys
sys.path.append(os.environ.get('SERIALBOX_ROOT') + '/python')
import serialbox as ser
import numpy as np
serializer = ser.Serializer(ser.OpenModeKind.Read, './data_single', 'HEAT_SINGLE')
sp = serializer.get_savepoint('starting_heat')
curr_T = serializer.read('init_T', sp[0])
scale     = serializer.read('scale', sp[0])[0]
t_steps   = serializer.read('t_steps', sp[0])[0]
nhalo   = serializer.read('nhalo', sp[0])[0]
full_shape = curr_T.shape
nx = full_shape[0] - 2 * nhalo
ny = full_shape[1] - 2 * nhalo
nz = full_shape[2] - 2
future_T = np.zeros((curr_T.shape[0],
                        curr_T.shape[1],
                        curr_T.shape[2]),dtype='float32')
for curr_step in range(t_steps):
    for ii in range(nhalo, nhalo + nx):  
        for jj in range(nhalo, nhalo + ny):
            for kk in range(1, nz +1):       
                future_T[ii,jj,kk] = curr_T[ii,jj,kk] \
                  + scale * (curr_T[ii-1,jj,   kk]   - 2.0*curr_T[ii,jj,kk] + curr_T[ii+1,jj,  kk]) \
                  + scale * (curr_T[ii,  jj-1, kk]   - 2.0*curr_T[ii,jj,kk] + curr_T[ii,  jj+1,kk]) \
                  + scale * (curr_T[ii,  jj,   kk-1] - 2.0*curr_T[ii,jj,kk] + curr_T[ii,  jj,  kk+1])
    curr_T[:,:,:] = future_T[:,:,:]
sp = serializer.get_savepoint('final_heat')
final_T = serializer.read('final_T', sp[0])

if np.allclose(final_T,curr_T):
    print("Solution is valid!")
    print('Max error:', np.max(np.abs(final_T - curr_T)))
else:
    raise Exception("Solution is NOT valid")